In [22]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
inputname="human.fasta"
outputname="human_PC_PseDNC_test"
path="data/"
propertyname="physical_chemical_properties_4.txt"

In [23]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [48]:
def extract_features_from_original_sequence(w,Lamda,num,pythysicalchemical_indices):
    phisical_chemical_proporties=pd.read_csv(path+propertyname,header=None,index_col=None)
    m6a_sequence=open(path+inputname)
    DNC_key=phisical_chemical_proporties.values[:,0]
    # DNC_key=np.array(['AA','AC','AG','AU','CA','CC','CG','CU','GA','GC','GG','GU','UA','UC','UG','UU'])
    DNC_value=phisical_chemical_proporties.values[:,num]
    DNC_value=np.array(DNC_value).T
    DNC_value_scale=[[]]*len(DNC_value)
    for i in xrange(len(DNC_value)):
        print "DNC_value[i]",DNC_value[i]
        print "type(DNC_value[i])",type(DNC_value[i])
        average_=sum(DNC_value[i]/len(DNC_value[i]))
        
        std_=np.std(DNC_value[i],ddof=1)
        DNC_value_scale[i]=[round((e-average_)/std_,2) for e in DNC_value[i]]
    DNC_value_scale=zip(*DNC_value_scale)
    DNC_len=len(DNC_value_scale[0])
    m6aseq=[]
    for line in m6a_sequence:
        if line.startswith('>'):
            pass
        else:
            m6aseq.append(line.replace('\n','').replace("\r",''))
    result_value=[]
    m6a_len=len(m6aseq[0])
    m6a_num=len(m6aseq)
    for m6a_line_index in xrange(m6a_num):
        frequency=[0]*16
        m6a_DNC_value=[[]]*(m6a_len-1)
        for m6a_line_doublechar_index in xrange(m6a_len):
            for DNC_index in xrange(16):
                if m6aseq[m6a_line_index][m6a_line_doublechar_index:m6a_line_doublechar_index+2]==DNC_key[DNC_index]:
                    m6a_DNC_value[m6a_line_doublechar_index]=DNC_value_scale[DNC_index]
                    frequency[DNC_index]+=1
        frequency=[e/float(sum(frequency)) for e in frequency]
        p=sum((frequency))
        #frequency=np.array(frequency)/float(sum(frequency))#(m6a_len-1)
        one_line_value_with = 0.0
        sita = [0] *  Lamda* pythysicalchemical_indices
        for lambda_index in xrange(1, Lamda* pythysicalchemical_indices+ 1):
            one_line_value_without_ = 0.0
            head_value=(lambda_index-1)%pythysicalchemical_indices
            for m6a_sequence_value_index in xrange(1, m6a_len - 3-(lambda_index-1)//pythysicalchemical_indices+1):    
                temp = round((np.array(m6a_DNC_value[m6a_sequence_value_index - 1][head_value])+
                                                             np.array(m6a_DNC_value[m6a_sequence_value_index ][head_value])),8)
                one_line_value_without_ += temp
            leng=(m6a_len - 3-(lambda_index-1)//pythysicalchemical_indices)
            one_line_value_without_ = round(one_line_value_without_ /leng ,8)
            sita[lambda_index - 1] = one_line_value_without_
            one_line_value_with += one_line_value_without_
        dim = [0] * (16 + Lamda* pythysicalchemical_indices)
        for index in xrange(1, 16 +  Lamda* pythysicalchemical_indices+1):
            if index <= 16:
                dim[index - 1] = frequency[index - 1] / (1.0 + w * one_line_value_with)
            else:
                dim[index - 1] = w * sita[index - 17] / (1.0 + w * one_line_value_with)
            dim[index-1]=round(dim[index-1],8)
        result_value.append(dim)
    pd.DataFrame(result_value).to_csv(path+outputname+str(w)+str(Lamda)+str(num)+".csv", header=None, index=None)
    print "savepath",path+outputname+str(w)+str(Lamda)+str(num)+".csv"

    m6a_sequence.close()

In [25]:
def train_svm(w,lamda,num):
    import time
    startTime=time.time()
    classifier="SVM"
    datapath =path+outputname+str(w)+str(lamda)+str(num)+".csv"
    print "datapath",datapath
    train_data = pd.read_csv(datapath, header=None, index_col=None)
    X = np.array(train_data)
    Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
    Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
    Y.extend(Y2)
    Y = np.array(Y)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12)
    ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
    pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+str(w)+str(lamda)+str(num)+'_predict.csv',header=None,index=False)
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata
    print X.shape[1]
    easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+str(w)+str(lamda)+str(num)+'.xls')
    endTime=time.time()
    elapsedTime=endTime-startTime
    print "end"
    print "elaspsed time:",elapsedTime
    return ACC

In [49]:
"""
    cross validation
"""
print "start"
ACC_=0
W=0
Lambda=0
for w in np.arange(0,1,0.1):
    for lamda in xrange(1,6):
        extract_features_from_original_sequence(w,lamda,list(np.arange(1,16)),15)#fetch features from original sequences
        ACC=train_svm(w,lamda,list(np.arange(1,16)))#construct a support vector machine
        if ACC>ACC_:
            ACC_=ACC
            W=w
            Lambda=lamda
print "W,Lambda",W,Lambda
print "ACC_",ACC_
save_list=[]
save_index=[]
for index in xrange(1,16):
    extract_features_from_original_sequence(W,Lambda,[index],1)#fetch features from original sequences
    ACC=train_svm(W,Lambda,[index])#construct a support vector machine
    save_list.append(ACC)
sorted_save_list=sorted(save_list)[::-1]
for data in sorted_save_list:
    save_index.append(save_list.index(data))
print "sorted_save_list",sorted_save_list
print "save_index",save_index
number_list=[]
temp_ACC=0
for number in save_index:
    number_list.append(number)
    print "number_list",number_list
    print "the length of number_list",len(number_list)
    extract_features_from_original_sequence(W,Lambda,number_list,len(number_list))#fetch features from original sequences
    ACC=train_svm(W,Lambda,number_list)#construct a support vector machine
    if ACC>=temp_ACC:
        temp_ACC=ACC
    else:
        print "W,Lambda,ACC",W,Lambda,temp_ACC
print "number_list",number_list
ACC_=0
W=0
Lambda=0
for w in np.arange(0,1,0.1):
    for lamda in xrange(1,6):
        extract_features_from_original_sequence(w,lamda,number_list,len(number_list))#fetch features from original sequences
        ACC=train_svm(w,lamda,list(np.arange(1,16)))#construct a support vector machine
        if ACC>ACC_:
            ACC_=ACC
            W=w
            Lambda=lamda
print "W,lambda,ACC",W,Lambda,ACC_
print "end"

start
DNC_value[i] [0.04 0.05 0.05 0.06 0.04 0.04 0.06 0.05 0.04 0.04 0.04 0.04 0.04 0.03 0.05
 0.04]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [0.06 0.07 0.07 0.07 0.06 0.08 0.07 0.1 0.06 0.06 0.06 0.06 0.06 0.07 0.07
 0.08]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [0.06 0.06 0.06 0.06 0.05 0.07 0.06 0.07 0.05 0.05 0.06 0.05 0.05 0.05 0.06
 0.07]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [2.99 4.27 4.21 6.8 3.47 6.69 6.8 9.61 2.71 2.0 2.99 3.47 2.0 1.85 4.27
 6.69]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [2.67 3.58 2.66 2.91 2.8 6.24 2.91 4.66 3.02 2.88 2.67 2.8 2.88 4.11 3.58
 6.24]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [14.25 18.41 17.31 21.98 17.48 21.34 21.98 24.79 14.66 14.51 14.25 17.48
 14.51 14.24 18.41 21.34]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [4.68 2.44 1.7 2.01 3.6 1.05 2.01 0.61 6.02 5.6 4.68 3.6 5.6 3.5 2.44 1.05]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [-0.77 1.44 0

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [51]:
save_index=xrange(1,16)
number_list=[]
temp_ACC=0
for number in save_index:
    number_list.append(number)
    print "number_list",number_list
    print "the length of number_list",len(number_list)
    extract_features_from_original_sequence(W,Lambda,number_list,len(number_list))#fetch features from original sequences
    ACC=train_svm(W,Lambda,number_list)#construct a support vector machine
    if ACC>=temp_ACC:
        temp_ACC=ACC
    else:
        print "W,Lambda,ACC",W,Lambda,temp_ACC
print "number_list",number_list
ACC_=0
W=0
Lambda=0
for w in np.arange(0,1,0.1):
    for lamda in xrange(1,6):
        extract_features_from_original_sequence(w,lamda,number_list,len(number_list))#fetch features from original sequences
        ACC=train_svm(w,lamda,list(np.arange(1,16)))#construct a support vector machine
        if ACC>ACC_:
            ACC_=ACC
            W=w
            Lambda=lamda
print "W,lambda,ACC",W,Lambda,ACC_
print "end"

number_list [1]
the length of number_list 1
DNC_value[i] [0.04 0.05 0.05 0.06 0.04 0.04 0.06 0.05 0.04 0.04 0.04 0.04 0.04 0.03 0.05
 0.04]
type(DNC_value[i]) <type 'numpy.ndarray'>
savepath data/human_PC_PseDNC_test0.01[1].csv
datapath data/human_PC_PseDNC_test0.01[1].csv
[[['SVMC:2.82842712475gamma:4.0', 0.67079646017699113, 0.7075268817204301, 0.5823008849557522, 0.5823008849557522, 0.7592920353982301, 0.6649333982830489, 0.63883495145631064, 0.63883495145631064, 0.34707234483245386, 0.67079646017699113, 658.0, 472.0, 272.0, 858.0, 1130.0, 1130.0]]]
17
end
elaspsed time: 17.6554250717
number_list [1, 2]
the length of number_list 2
DNC_value[i] [0.04 0.05 0.05 0.06 0.04 0.04 0.06 0.05 0.04 0.04 0.04 0.04 0.04 0.03 0.05
 0.04]
type(DNC_value[i]) <type 'numpy.ndarray'>
DNC_value[i] [0.06 0.07 0.07 0.07 0.06 0.08 0.07 0.1 0.06 0.06 0.06 0.06 0.06 0.07 0.07
 0.08]
type(DNC_value[i]) <type 'numpy.ndarray'>
savepath data/human_PC_PseDNC_test0.01[1, 2].csv
datapath data/human_PC_PseDNC_test